# Comparing LLM Path Extractors for Knowledge Graph Construction

In this notebook, we'll compare three different LLM Path Extractors from llama_index:
1. SimpleLLMPathExtractor
2. SchemaLLMPathExtractor
3. DynamicLLMPathExtractor (New)

We'll use a Wikipedia page as our test data and visualize the resulting knowledge graphs using Pyvis.

## Setup and Imports

In [1]:
!pip install llama_index pyvis wikipedia

In [2]:
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

import wikipedia

import os

In [3]:
import nest_asyncio

nest_asyncio.apply()

## Set up LLM Backend

In [4]:
from utils import load_env

load_env()
# Set up global configurations
llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")

Settings.llm = llm
Settings.chunk_size = 2048
Settings.chunk_overlap = 20

## Fetch Some Raw Text from Wikipedia

In [5]:
def get_wikipedia_content(title):
    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation page. Options: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"Page '{title}' does not exist.")
    return None

In [6]:
wiki_title = "Barack Obama"
content = get_wikipedia_content(wiki_title)

if content:
    document = Document(text=content, metadata={"title": wiki_title})
    print(
        f"Fetched content for '{wiki_title}' (length: {len(content)} characters)"
    )
else:
    print("Failed to fetch Wikipedia content.")

Fetched content for 'Barack Obama' (length: 84410 characters)


In [12]:
len(document.text)

84410

In [13]:
document.metadata

{'title': 'Barack Obama'}

## 1. SimpleLLMPathExtractor

In [7]:
kg_extractor = SimpleLLMPathExtractor(
    llm=llm, max_paths_per_chunk=20, num_workers=4
)

simple_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

simple_index.property_graph_store.save_networkx_graph(
    name="./SimpleGraph.html"
)
simple_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

/opt/anaconda3/envs/latest_llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting paths from text: 100%|██████████| 11/11 [00:10<00:00,  1.03it/s]


[(EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'd3697893-86de-4acb-9088-eeb6f4e42518'}, name='Obama'),
  Relation(label='Enrolled at', source_id='Obama', target_id='Harvard law school', properties={'title': 'Barack Obama', 'triplet_source_id': '53f86eaf-2614-411f-97d6-75fba030664d'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '53f86eaf-2614-411f-97d6-75fba030664d'}, name='Harvard law school')),
 (EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'd3697893-86de-4acb-9088-eeb6f4e42518'}, name='Obama'),
  Relation(label='Met with', source_id='Obama', target_id='Mahmoud abbas', properties={'title': 'Barack Obama', 'triplet_source_id': 'ce352188-5f95-40df-a2b8-aac18d80dcca'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'ce352188-5f95-40df-a2b8-aac18d80dcca'}, name='M

## 2. DynamicLLMPathExtractor

### Without intial ontology :
Here, we let the LLM define the ontology on the fly, giving it full freedom to label the nodes as it best sees fit.

In [15]:
document.text

'Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American  president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004. \nObama was born in Honolulu, Hawaii. He graduated from Columbia University in 1983 with a Bachelor of Arts degree in political science and later worked as a community organizer in Chicago. In 1988, Obama would enroll in Harvard Law School, where he became the first black president of the Harvard Law Review. He became a civil rights attorney and an academic, teaching constitutional law at the University of Chicago Law School from 1992 to 2004. He also went into elective politics; Obama represented the 13th district in the Illinois Senate from 1997 until 2004, when he successfully ran for the U.S. Senate. In the 20

In [16]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    # Let the LLM infer entities and their labels (types) on the fly
    allowed_entity_types=None,
    # Let the LLM infer relationships on the fly
    allowed_relation_types=None,
    # LLM will generate any entity properties, set `None` to skip property generation (will be faster without)
    allowed_relation_props=[],
    # LLM will generate any relation properties, set `None` to skip property generation (will be faster without)
    allowed_entity_props=[],
)

dynamic_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index.property_graph_store.save_networkx_graph(
    name="./DynamicGraph.html"
)

dynamic_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 47.31it/s]
Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:42<00:00,  3.86s/it]


[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'cef51d95-b53e-4464-a8e6-c636f78b4449'}, name='Barack Obama'),
  Relation(label='ATTENDED', source_id='Barack Obama', target_id='University of Chicago Laboratory Schools', properties={'title': 'Barack Obama', 'triplet_source_id': '6321d699-e004-4a2f-ac74-ba66615a489b'}),
  EntityNode(label='EDUCATIONAL_INSTITUTION', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '6321d699-e004-4a2f-ac74-ba66615a489b'}, name='University of Chicago Laboratory Schools')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'cef51d95-b53e-4464-a8e6-c636f78b4449'}, name='Barack Obama'),
  Relation(label='WARNED', source_id='Barack Obama', target_id='actions could  harden Palestinian attitudes for generations ', properties={'warning_target': 'Israel', 'title': 'Barack Obama', 'triplet_source_id': 'cef51d95-b53e-4464-a8e6-c636f78b4

In [14]:
dynamic_index.property_graph_store.get_triplets()[:5]

[]

### With initial ontology for guided KG extraction : 
Here, we have partial knowledge of what we want to detect, we know the article is about Barack Obama, so we define some entities and relations that could help guide the LLM in the labeling process as it detects the entities and relations. This doesn't guarantee that the LLM will use them, it simply guides it and gives it some ideas. It will still be up to the LLM to decide whether it uses the entities and relations we provide or not. 

In [9]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=["POLITICIAN", "POLITICAL_PARTY"],
    allowed_relation_types=["PRESIDENT_OF", "MEMBER_OF"],
    allowed_relation_props=["description"],
    allowed_entity_props=["description"],
)

dynamic_index_2 = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index_2.property_graph_store.save_networkx_graph(
    name="./DynamicGraph_2.html"
)
dynamic_index_2.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 44.84it/s]
Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:39<00:00,  3.61s/it]


[(EntityNode(label='POLITICIAN', embedding=None, properties={'description': '44th President of the United States', 'title': 'Barack Obama', 'triplet_source_id': '38f5ad9d-0190-4a26-8179-ac1297e42e1c'}, name='Barack Obama'),
  Relation(label='ATTENDED', source_id='Barack Obama', target_id='University of Chicago Laboratory Schools', properties={'title': 'Barack Obama', 'triplet_source_id': '892906b8-faf9-4e34-a78f-bdb35d7a92cf'}),
  EntityNode(label='SCHOOL', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '892906b8-faf9-4e34-a78f-bdb35d7a92cf'}, name='University of Chicago Laboratory Schools')),
 (EntityNode(label='POLITICIAN', embedding=None, properties={'description': '44th President of the United States', 'title': 'Barack Obama', 'triplet_source_id': '3f24a965-6353-4505-8ea4-4253be69d109'}, name='Obama'),
  Relation(label='CALLED', source_id='Obama', target_id='war in Iraq', properties={'description': 'called the war in Iraq a "dangerous distraction"', 'titl

# 3 - SchemaLLMPathExtractor

In [10]:
kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    strict=False,  # Set to False to showcase why it's not going to be the same as DynamicLLMPathExtractor
    possible_entities=None,  # USE DEFAULT ENTITIES (PERSON, ORGANIZATION... etc)
    possible_relations=None,  # USE DEFAULT RELATIONSHIPS
    possible_relation_props=[
        "extra_description"
    ],  # Set to `None` to skip property generation
    possible_entity_props=[
        "extra_description"
    ],  # Set to `None` to skip property generation
    num_workers=4,
)

schema_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

schema_index.property_graph_store.save_networkx_graph(
    name="./SchemaGraph.html"
)
schema_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Extracting paths from text with schema:  91%|█████████ | 10/11 [02:30<00:21, 21.12s/it]

InternalServerError: Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_6fe0543f8e44e7a4b3623a1d131f63f2 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}

## Comparison and Analysis

Let's compare the results of the three extractors:

1. **SimpleLLMPathExtractor**: This extractor creates a basic knowledge graph without any predefined schema. It may produce a larger number of diverse relationships but might lack consistency in entity and relation naming.


3. **DynamicLLMPathExtractor**: 
    - This new extractor combines the flexibility of SimpleLLMPathExtractor with some initial guidance from a schema. It can expand beyond the initial entity and relation types, potentially producing a rich and diverse graph while maintaining some level of consistency. 
    - Not giving it any entities or relations to start with in the input gives the LLM complete freedom to infer the schema on the fly as it best sees fit. This is going to vary based on the LLM and the temperature used.

3. **SchemaLLMPathExtractor**: With a predefined schema, this extractor produces a more structured graph. The entities and relations are limited to those specified in the schema, which can lead to a more consistent but potentially less comprehensive graph. Even if we set "strict" to false, the extracted KG Graph doesn't reflect the LLM's pursuit of trying to find new entities and types that fall outside of the input schema's scope.


## Key observations:

- The SimpleLLMPathExtractor graph might have the most diverse set of entities and relations.
- The SchemaLLMPathExtractor graph should be the most consistent but might miss a lot of relationships that don't fit the predefined schema, even if we don't impose a strict validation of the schema.
- The DynamicLLMPathExtractor graph should show a balance between diversity and consistency, potentially capturing important relationships that the schema-based approach might miss while still maintaining some structure.

## The choice between these extractors depends on the specific use case:

- Use SimpleLLMPathExtractor for exploratory analysis where you want to capture a wide range of potential relationships for RAG applications, without caring about the entity types.
- Use SchemaLLMPathExtractor when you have a well-defined domain and want to ensure consistency in the extracted knowledge.
- Use DynamicLLMPathExtractor when you want a balance between structure and flexibility, allowing the model to discover new entity and relation types while still providing some initial guidance. This one is especially useful if you want a KG with labeled (typed) entities but don't have an input Schema (or you've partially defined the schema as a starting base).